<a href="https://colab.research.google.com/github/felipefernandes/lisa/blob/main/LiSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -q -U google-generativeai
!pip install -q -U pathlib

In [5]:
import pathlib
import hashlib
import google.generativeai as genai
from google.colab import userdata, drive
import pandas as pd
import numpy as np
from IPython.display import display
from IPython.display import Markdown

In [8]:
# Configuração do GenerativeAI
GOOGLE_API_KEY = userdata.get('GEMINIKEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [11]:
# Listando modelos disponíveis
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [64]:
# Documentos

estruturas = [
    {"nome": "Impromptu Networking", "duracao": "5-20 min", "proposito": "Compartilhe rapidamente os desafios e as expectativas, crie novas conexões", "label": "Disseminar"},
    {"nome": "9 Whys", "duracao": "5-20 min", "proposito": "Torne claro o propósito do trabalho juntos", "label": "Analisar"},
    {"nome": "What, So What, Now What", "duracao": "15-45 min", "proposito": "Analisar juntos o progresso até então e decidir quais ajustes são necessários", "label": "Analisar"},
    {"nome": "TRIZ", "duracao": "30-45 min", "proposito": "Pare atividades e comportamentos contraproducentes para abrir espaço para a inovação", "label": "Revelar"},
    {"nome": "Appreciative Interviews", "duracao": "30-60 min", "proposito": "Descobrindo e aproveitando as causas-raiz do sucesso", "label": "Revelar, Disseminar"},
    {"nome": "1-2-4-All", "duracao": "10-12 min", "proposito": "Engage todos simultaneamente na geração de perguntas, ideias e sugestões.", "label": "Revelar"},
    {"nome": "User Experience Fishbowl", "duracao": "25-70 min", "proposito": "Compartilhe com uma comunidade maior a capacidade obtida a partir da experiência", "label": "Disseminar"},
    {"nome": "15% Solutions", "duracao": "15-20 min", "proposito": "Descubra e concentre-se no que cada pessoa tem a liberdade e os recursos para fazer agora", "label": "Revelar"},
    {"nome": "25-to-10 Crowdsourcing", "duracao": "20-30 min", "proposito": "Gerar e analisar rapidamente as ideias factíveis mais poderosas de um grupo", "label": "Revelar"},
    { "nome": "Troika Consulting", "duracao": "15-30 min", "proposito": "Obtenha ajuda prática e criativa dos colegas imediatamente", "label": "Ajuda, Revelar"},
    { "nome": "Conversation Café", "duracao": "35-60 min", "proposito": "Engage todos na compreensão de grandes desafios", "label": "Disseminar"},
    { "nome": "Min Specs", "duracao": "20-50 min", "proposito": "Especifique apenas os absolutos “Devemos” e “Não Devemos” para alcançar um propósito", "label": "Estratégia"},
    { "nome": "Wise Crowds", "duracao": "10-60 min. per person", "proposito": "Acesse a sabedoria do grupo inteiro em ciclos rápidos", "label": "Ajuda, Revelar"},
    { "nome": "Wicked Questions", "duracao": "20 min", "proposito": "Articule os desafios paradoxais que um grupo deve enfrentar para ter sucesso", "label": "Revelar"},
    { "nome": "Drawing Together", "duracao": "30-40 min", "proposito": "Revele ideias e caminhos para avançar através da expressão não-verbal", "label": "Revelar"},
    { "nome": "Improv Prototyping", "duracao": "15-20 min. per round", "proposito": "Desenvolva soluções eficazes para desafios crônicos, enquanto se diverte seriamente", "label": "Revelar"},
    { "nome": "Agreement-Certainty Matrix", "duracao": "30-45 min", "proposito": "Classifique os desafios nos domínios simples, complicado, complexo e caótico", "label": "Análise"},
    { "nome": "Shift & Share", "duracao": "35-90 min", "proposito": "Dissemine boas ideias e faça conexões informais com inovadores", "label": "Disseminar"},
    { "nome": "Heard, Seen, Respected", "duracao": "25 min", "proposito": "Pratique a escuta profunda e empatia com colegas", "label": "Ajuda"},
    { "nome": "Social Network Webbing", "duracao": "45-60 min", "proposito": "Mapeie conexões informais e decida como fortalecer a rede para alcançar um propósito", "label": "Ajuda, Disseminar"},
    { "nome": "Design Storyboards", "duracao": "25-70 min", "proposito": "Defina os elementos passo-a-passo para conduzir reuniões a finais produtivos", "label": "Planejar"},
    { "nome": "Open Space", "duracao": "90 min. to 3 days", "proposito": "Libere a ação e liderança inerentes a grupos de qualquer tamanho", "label": "Revelar"},
    { "nome": "Discovery & Action Dialogue", "duracao": "25-70 min", "proposito": "Descubra, invente e desencadeie soluções locais para problemas crônicos", "label": "Revelar"},
    { "nome": "Integrated Autonomy", "duracao": "60-80 min", "proposito": "Mude de soluções 'ou-um-ou-outro' para robustas soluções 'ambos-e'", "label": "Estratégia"},
    { "nome": "Generative Relationships", "duracao": "25 min", "proposito": "Revele padrões de relacionamento que criam surpreendente valor ou disfunções", "label": "Análise"},
    { "nome": "Critical Uncertainties", "duracao": "60-100 min", "proposito": "Desenvolva estratégias para operar em uma variedade de futuros plausíveis porém imprevisíveis", "label": "Estratégia"},
    { "nome": "Purpose-to-Practice", "duracao": "25-120 min", "proposito": "Desenhe os cinco elementos essenciais para uma iniciativa resiliente e duradoura", "label": "Planejar"},
    { "nome": "Ecocycle Planning", "duracao": "60-95 min", "proposito": "Analise o portfólio completo de atividades e relacionamentos para identificar obstáculos e oportunidades para o progresso", "label": "Análise, Estratégia"},
    { "nome": "Panarchy", "duracao": "1-2 hr", "proposito": "Entenda como sistemas interconectados interagem, evoluem, disseminam inovação e transformam", "label": "Estratégia"},
    { "nome": "What I Need From You", "duracao": "45-70 min", "proposito": "Faça emergir as necessidades essenciais entre áreas e aceite ou rejeite pedidos de suporte", "label": "Ajuda"},
    { "nome": "Celebrity Interview", "duracao": "25-60 min", "proposito": "Reconectar a experiência de líderes e especialistas com pessoas mais próximas do desafio em mãos", "label": "Disseminar"},
    { "nome": "Helping Heuristics", "duracao": "15 min", "proposito": "Pratique métodos progressivos para ajudar os outros, receber ajuda e pedir ajuda", "label": "Ajuda"},
    { "nome": "Simple Ethnography", "duracao": "1-6 hr", "proposito": "Observe e registre os comportamentos reais dos usuários em campo", "label": "Análise, Revelar"}
]

solucoes = [
    {"solucao": "Revelar, gerar aprimorar idéias ou soluções"},
    {"solucao": "Compartilhar ou disseminar idéias, conhecimentos ou experiências"},
    {"solucao": "Analisar, diagnosticar ou processar"},
    {"solucao": "Pedir ou oferecer ajuda"},
    {"solucao": "Elaborar estratégias"},
    {"solucao": "Planejar"}
]

In [ ]:
df = pd.DataFrame(estruturas)
df.columns = ['nome', 'duracao', 'proposito', 'label']
df

In [ ]:
# FAzer um dataframe pra solucoes,
# fazer uma funcao para adicionar embeds
# fazer uma funcao para verificar se o embed da solucao, tem alguma relacao com a lista de estruturas
# atualizar a funcao de consutla principal para identificar solucao(oes) compativeis com o contexto fornecido pelo usuário <input>

In [15]:
# Inicializando o modelo
model = "models/embedding-001"

In [69]:
# Função para adicionar colunas de Embeds
def addEmbeddings(nome, proposito, duracao, label):
  return genai.embed_content(model=model, content=proposito, title=nome, task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [ ]:
# Adicionando Embeddings no DataFrame
df["Embeddings"] = df.apply(lambda row: addEmbeddings(row["nome"], row["proposito"], row["duracao"], row["label"]), axis=1)
df

In [71]:
# Função para gerar e buscar e consultar
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model, content=consulta, task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(df['Embeddings']), embedding_da_consulta)

  limite_relevancia = 0.7 # Definindo o limite da relevancia
  indices_relevantes = np.where(produtos_escalares >= limite_relevancia)[0]

  resultados_relevantes = []
  for indice in indices_relevantes:
    resultados_relevantes.append(df.iloc[indice]['nome'])

  if resultados_relevantes:
    return resultados_relevantes # Exibe uma lista de resultados relevantes
  else:
    indice = np.argmax(produtos_escalares)
    return df.iloc[indice]['nome'] # garante que ao menos 1 resultado seja exibido

In [74]:
# Testando consultas baseadas em contextos
consulta1 = "Time experiente, que precisa de novas idéias para um projeto"
consulta2 = "Buscar a causa raiz de problemas"
consulta3 = "O time precisa melhorar suas relações"

trecho = gerar_e_buscar_consulta(consulta3, df, model)
print(trecho)

What, So What, Now What


In [36]:
# Preparando e instanciando o modo generativo para respostas
model_generative_config = {
    "temperature": 2,
    "candidate_count": 1,
    "top_p": 0.95,
    "top_k": 0,
    "max_output_tokens": 8192,
}

model_generative_safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

system_instruction = "Seu nome é Lisa. Uma assistente virtual, com habilidades de um Coach profissional e experiente facilitadora de reuniões e atividades presenciais ou online com amplo conhecimento de aplicação de Estruturas Libertadoras"

model_generative = genai.GenerativeModel("gemini-1.5-pro-latest",
                                         generation_config=model_generative_config,
                                         system_instruction=system_instruction,
                                         safety_settings=model_generative_safety_settings)

In [39]:
chat = model_generative.start_chat(history=[
    {
      "role": "user",
      "parts": ["Se apresente."]
    },
])

In [41]:
# Teste de iteração
chat.send_message("Se apresente")
print(chat.last.text)

Olá! 👋 Eu sou Lisa, sua assistente virtual. 

Pense em mim como um coach profissional, sempre pronta para te ajudar a alcançar seus objetivos, trabalhando lado a lado com você.  

Sou também uma facilitadora experiente, capaz de conduzir reuniões e atividades, tanto presenciais quanto online, de forma envolvente e produtiva, sempre buscando o melhor resultado para todos.

Meu diferencial? Domino as Estruturas Libertadoras, uma metodologia inovadora que estimula a criatividade, colaboração e engajamento em grupos, tornando o trabalho mais leve e eficaz. 

O que posso fazer por você hoje? 😊 



In [29]:
# Prompting

prompt = f"Crie um roteiro usando a Estrutura Libertadora {trecho} para o contexto"

prompt

# response = model_generative.generate_content(prompt)
# print(response.text)

'Crie um roteiro usando a Estrutura Libertadora Appreciative Interviews para o contexto'

In [62]:
# @TODO
# O usuário deve fornecer um contexto
# LISA precisa identificar, dentro do contexto quais sequencias fazem sentido. (WIP)
# Montar e exibir um Roteiro com a estrutura: Título do Roteiro / Duração (soma do total de atividades min-max) / Abertura / Colhendo dados / Brainstorming de açoes / Fechamento

In [45]:
contexto1 = "Tenho um time recém criado que passou por muitas mudanças e seus membros precisam se reconectar uns com os outros"
contexto2 = "Tenho um time experiente que apesar do bom clima interperssoal não expõem seus problemas de maneira clara e construtiva"
contexto3 = "Tenho um time experiente que precisa fazer melhorias no seu processo de trabalho para realizar entregas importantes num futuro breve"